In [1]:
import os
import glob

il_folders = os.listdir("data")
il_folders

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kingo\AppData\Local\Temp\ipykernel_36368\2774585212.py", line 4, in <module>
    il_folders = os.listdir("data")
FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'data'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-package

In [4]:
datas = []
def refactor_data(il_folders):
    for folder in il_folders:
        results = glob.glob(f"data/{folder}/*/*/*.jpg")
        print(results)
        datas.append(results)

refactor_data(il_folders)

datas

[]
[]
['data/Hepatitis\\Negative\\iPhone 11\\IMG_8437.JPG', 'data/Hepatitis\\Negative\\iPhone 11\\IMG_8438.JPG', 'data/Hepatitis\\Negative\\iPhone 11\\IMG_8439.JPG', 'data/Hepatitis\\Negative\\iPhone 11\\IMG_8440.JPG', 'data/Hepatitis\\Negative\\iPhone 11\\IMG_8441.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1249.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1250.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1251.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1252.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1253.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1254.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1255.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1256.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1257.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1258.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1259.JPG', 'data/Hepatitis\\Negative\\iPhone 6\\IMG_1260.JPG', 'data/Hepatitis\\Positive\\iPhone 6\\FullSizeRender.jpg', 'data/Hepatitis\\Positive\\iPhone 6\\IMG_1262.

[[],
 [],
 ['data/Hepatitis\\Negative\\iPhone 11\\IMG_8437.JPG',
  'data/Hepatitis\\Negative\\iPhone 11\\IMG_8438.JPG',
  'data/Hepatitis\\Negative\\iPhone 11\\IMG_8439.JPG',
  'data/Hepatitis\\Negative\\iPhone 11\\IMG_8440.JPG',
  'data/Hepatitis\\Negative\\iPhone 11\\IMG_8441.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1249.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1250.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1251.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1252.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1253.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1254.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1255.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1256.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1257.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1258.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1259.JPG',
  'data/Hepatitis\\Negative\\iPhone 6\\IMG_1260.JPG',
  'data/Hepatitis\\Positive\\iPhone 6\\FullSizeRender.jpg',
  'data

In [10]:
from ultralytics import YOLO
model = YOLO("../weights/best_type.pt")
model.export(format="onnx")


Ultralytics YOLOv8.0.53  Python-3.9.0 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from ..\weights\best_type.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.9 MB)

ONNX: starting export with onnx 1.14.0...
ONNX: export success  0.9s, saved as ..\weights\best_type.onnx (11.7 MB)

Export complete (1.3s)
Results saved to C:\Users\kingo\OneDrive\ \work\Make Sure\workhorse\server\weights
Predict:         yolo predict task=detect model=..\weights\best_type.onnx imgsz=640 
Validate:        yolo val task=detect model=..\weights\best_type.onnx imgsz=640 data=types.yaml 
Visualize:       https://netron.app


'..\\weights\\best_type.onnx'

In [22]:
from PIL import Image
import onnxruntime as ort
import numpy as np


def detect_objects_on_image(buf):
    input, img_width, img_height = prepare_input(buf)
    output = run_model(input)
    return process_output(output,img_width,img_height)

def prepare_input(img):
    img_width, img_height = img.size
    img = img.resize((640, 640))
    img = img.convert("RGB")
    input = np.array(img)
    input = input.transpose(2, 0, 1)
    input = input.reshape(1, 3, 640, 640) / 255.0
    return input.astype(np.float32), img_width, img_height

def run_model(input):
    model = ort.InferenceSession("../weights/best_type.onnx")
    outputs = model.run(["output0"], {"images":input})
    return outputs[0]

def iou(box1,box2):
    return intersection(box1,box2)/union(box1,box2)

def union(box1,box2):
    box1_x1,box1_y1,box1_x2,box1_y2 = box1[:4]
    box2_x1,box2_y1,box2_x2,box2_y2 = box2[:4]
    box1_area = (box1_x2-box1_x1)*(box1_y2-box1_y1)
    box2_area = (box2_x2-box2_x1)*(box2_y2-box2_y1)
    return box1_area + box2_area - intersection(box1,box2)

def intersection(box1,box2):
    box1_x1,box1_y1,box1_x2,box1_y2 = box1[:4]
    box2_x1,box2_y1,box2_x2,box2_y2 = box2[:4]
    x1 = max(box1_x1,box2_x1)
    y1 = max(box1_y1,box2_y1)
    x2 = min(box1_x2,box2_x2)
    y2 = min(box1_y2,box2_y2)
    return (x2-x1)*(y2-y1)

yolo_classes = [
    "HIV", "Hepatitis", "Syphilis"
]

def process_output(output, img_width, img_height):
    output = output[0].astype(float)
    output = output.transpose()


    boxes = []
    for row in output:
        prob = row[4:].max()
        if prob < 0.5:
            continue
        class_id = row[4:].argmax()
        label = yolo_classes[class_id]

        xc, yc, w, h = row[:4]
        x1 = (xc - w/2) / 640 * img_width
        y1 = (yc - h/2) / 640 * img_height
        x2 = (xc + w/2) / 640 * img_width
        y2 = (yc + h/2) / 640 * img_height
        boxes.append([x1, y1, x2, y2, label, prob])

    boxes.sort(key=lambda x: x[5], reverse=True)
    result = []
    while len(boxes) > 0:
        result.append(boxes[0])
        boxes = [box for box in boxes if iou(box, boxes[0]) < 0.7]
    return result

from pillow_heif import register_heif_opener

register_heif_opener()


image = Image.open('./tests/IMG_5652.HEIC')

detect_objects_on_image(image)


[[1194.0241762161256,
  1423.067678833008,
  1341.1451002120973,
  1668.7142303466799,
  'Hepatitis',
  0.707928478717804]]

In [34]:
import cv2
import numpy as np
from kraken import binarization

from pyzbar.pyzbar import decode
from pyzbar.pyzbar import ZBarSymbol

from pillow_heif import register_heif_opener
from PIL import Image

register_heif_opener()


img = Image.open('tests/IMG_1345.JPG')
qcd = cv2.QRCodeDetector()

# bw_im = binarization.nlbin(img)

# retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(bw_im)
# print(decoded_info)
# decode(bw_im, symbols=[ZBarSymbol.QRCODE])

decoded_info = decode(img, symbols=[ZBarSymbol.QRCODE])[0].data.decode(encoding='utf-8')
decoded_info


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kingo\AppData\Local\Temp\ipykernel_4220\3511108124.py", line 23, in <module>
    decoded_info = decode(img, symbols=[ZBarSymbol.QRCODE])[0].data.decode(encoding='utf-8')
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib

In [54]:
import cv2
from pyzbar.pyzbar import decode
from pyzbar.pyzbar import ZBarSymbol

im = cv2.imread('tests/IMG_8442.JPG', cv2.IMREAD_GRAYSCALE)

clean_im = cv2.medianBlur(im, 25) 
ret, bw_im = cv2.threshold(clean_im, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

barcodes = decode(bw_im, symbols=[ZBarSymbol.QRCODE])
print(f'barcodes: {barcodes}')

barcodes: []


In [62]:
from qreader import QReader
import cv2

from pillow_heif import register_heif_opener
from PIL import Image

register_heif_opener()


img = Image.open('tests/IMG_5652.HEIC')

detector = QReader()
# img = cv2.cvtColor(cv2.imread('tests/IMG_8437.JPG'), cv2.COLOR_BGR2RGB)
# Detect and Decode the QR
print(detector.detect_and_decode(image=convert_to_cv2(img)))

Fusing layers... 
IDetect.fuse
('0309b9f8-c568-49de-9dd4-81134905e934\r\nfaff25ed-c27d-42f5-b67a-909c933450de',)


In [1]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# Train the model
model.train(data='t_res.yaml', epochs=30, imgsz=640)

New https://pypi.org/project/ultralytics/8.0.124 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.53  Python-3.9.0 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=t_res.yaml, epochs=30, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=F

In [65]:
from ultralytics.yolo.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model='../weights/best_type.pt', imgsz=640, half=False, device="cpu")


image 1/1 C:\Users\kingo\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 (no detections), 169.2ms
Speed: 2.2ms preprocess, 169.2ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.53  Python-3.9.0 torch-1.13.1+cpu CPU
val: Scanning D:\SKB\ETC\Schoolbase\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:30<00:00,  4.26it/s]
                   all        128        929   9.94e-05    0.00895   0.000124   5.45e-05
Speed: 3.3ms preprocess, 208.9ms inference, 0.0ms loss, 3.8ms postprocess per image
Ultralytics YOLOv8.0.53  Python-3.9.0 torch-1.13.1+cpu CPU

PyTorch: starting from ..\weights\best_type.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.9 MB)

TorchScript: starting export with torc

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kingo\AppData\Local\Temp\ipykernel_4220\1485718298.py", line 4, in <module>
    benchmark(model='../weights/best_type.pt', imgsz=640, half=False, device="cpu")
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\ultralytics\yolo\utils\benchmarks.py", line 86, in benchmark
    results = export.val(data=data, batch=1, imgsz=imgsz, plots=False, device=device, half=half, verbose=False)
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\autograd\grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\ultralytics\yolo\engine\model.py", line 256, in val
    validator(model=self.model)
  File 

In [23]:
import cv2
from ultralytics import YOLO

from pillow_heif import register_heif_opener

from PIL import Image

def convert_to_cv2(img):
    open_cv_image = np.array(img) 
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    
    return open_cv_image

register_heif_opener()

img = Image.open('tests/IMG_5652.HEIC')

image = convert_to_cv2(img)

model = YOLO('../weights/best_type.pt')

res = model.predict(source=image)


0: 640x480 1 Hepatitis, 131.7ms
Speed: 1.0ms preprocess, 131.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [42]:
import cv2

img = cv2.imread('tests/IMG_8450.JPG')

img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

img = cv2.inRange(img, (150, 60, 100), (255, 255, 255))

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11,11))

img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, 6)

contours, hier = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

print('number of lines in test is ', len(contours)/2 )

number of lines in test is  13.0


In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [55]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets/t_res/",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 112 files belonging to 2 classes.
Using 90 files for training.
Using 22 files for validation.


In [47]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [48]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [49]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [56]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/25
1/1 [==============================] - 20s 20s/step - loss: 0.3183 - accuracy: 0.9111 - val_loss: 0.7292 - val_accuracy: 0.3636
Epoch 2/25
1/1 [==============================] - 16s 16s/step - loss: 0.6386 - accuracy: 0.7556 - val_loss: 0.7436 - val_accuracy: 0.3636
Epoch 3/25
1/1 [==============================] - 16s 16s/step - loss: 0.3637 - accuracy: 0.8111 - val_loss: 0.7421 - val_accuracy: 0.3636
Epoch 4/25
1/1 [==============================] - 16s 16s/step - loss: 0.2867 - accuracy: 0.8889 - val_loss: 0.7371 - val_accuracy: 0.3636
Epoch 5/25
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kingo\AppData\Local\Temp\ipykernel_3092\2660810289.py", line 11, in <module>
    model.fit(
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1564, in fit
    tmp_logs = self.train_function(iterator)
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\kingo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\eager\def_function.py", line 915, in __call_

In [54]:
img = keras.utils.load_img(
    "datasets/t_res/P/IMG_1290.JPG", target_size=image_size
)
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(predictions[0])
print(f"This image is {100 * (1 - score):.2f}% N and {100 * score:.2f}% P.")

1/1 [==============================] - 0s 72ms/step
This image is 54.96% N and 45.04% P.
